In [ ]:
import sqlite3
import time

import numpy as np
import torch
from PIL import Image
from torchvision.models import MobileNet_V2_Weights
from torchvision.transforms import transforms

from queue import PriorityQueue

model = torch.hub.load('pytorch/vision:v0.16.0', 'mobilenet_v2', weights=MobileNet_V2_Weights.DEFAULT)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
model = model.to(device)


def init_main_db():
    conn = sqlite3.connect('pixel_cache_g.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pixel_results (
            x INT,
            y INT,
            r INT,
            g INT,
            b INT,
            rG REAL,
            gG REAL,
            bG REAL,
            rO INT,
            gO INT,
            bO INT,
            S REAL,
            apple REAL,
            PRIMARY KEY (x, y, r, g, b)
        )
    ''')
    conn.commit()
    conn.close()


init_main_db()


def get_db_connection():
    """Creates and returns a database connection."""
    return sqlite3.connect('pixel_cache_g.db')


def get_prediction_from_db(pixel):
    """Retrieves the prediction and gradients for a specific pixel from the database."""
    # Ensure all elements are standard Python integers
    x, y, r, g, b = map(lambda e: e if isinstance(e, int) else e.item(), pixel[:5])
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT apple, rG, gG, bG FROM pixel_results WHERE x=? AND y=? AND r=? AND g=? AND b=?
        ''', (x, y, r, g, b))
        result = cursor.fetchone()
    return result


def insert_prediction_into_db(pixel):
    """Inserts a new prediction and its gradients into the database, or updates the existing entry if there's a conflict."""
    # Convert tensor elements to standard Python types (int for RGB values and float for gradients)
    x, y, r, g, b, rG, gG, bG, rO, gO, bO, S, apple = map(lambda e: e if isinstance(e, (int, float)) else e.item(), pixel)
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('''INSERT OR IGNORE INTO pixel_results (x, y, r, g, b, rG, gG, bG,rO, gO, bO, S, apple) VALUES
        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''' , (x, y, r, g, b, rG, gG, bG, rO, gO, bO, S, apple))
        conn.commit()


def predict(pixel, image, model):
    db_result = get_prediction_from_db(pixel)
    if db_result:
        return list(pixel[:5]) + list(db_result) + [1337]
    else:
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        image_tensor = preprocess(image).unsqueeze(0).to(device)

        x, y, r, g, b = pixel[:5]

        image_tensor[:, :, y, x] = torch.tensor([r / 255., g / 255., b / 255.], device=device)

        image_tensor.requires_grad_(True)

        model.zero_grad()
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        apple_prob = probabilities[0, 948].item()

        # now = titme.time()
        # output[0, 948].backward()
        # print(time.time() - now)
        # gradients = image_tensor.grad[0, :, y, x].tolist()

        # sum_gradient = sum([abs(g) for g in gradients])
        # new_pixel = lis(pixel[:5]) + gradients + list(pixel[8:11]) + [sum_gradient] + [apple_prob]
        new_pixel = list(pixel[:5]) + [0,0,0] + list(pixel[8:11]) + [0] + [apple_prob]

        insert_prediction_into_db(new_pixel)

        return new_pixel


def get_top_pixels_by_gradient_sum_sorted(image, model, top_x=100):
    model.eval()
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_tensor = preprocess(image).unsqueeze(0).to(device)
    image_tensor.requires_grad_(True)
    output = model(image_tensor)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    target_class_prob = probabilities[0, 948]
    model.zero_grad()
    target_class_prob.backward()
    gradients = image_tensor.grad.data[0]
    abs_gradients_sum = gradients.abs().sum(dim=0)
    indices = torch.topk(abs_gradients_sum.view(-1), top_x).indices
    top_pixels = [(idx // image_tensor.size(3), idx % image_tensor.size(3)) for idx in indices]
    top_pixels_info = []
    for x, y in top_pixels:
        pixel_vals = image_tensor[0, :, y, x] * torch.tensor([0.229, 0.224, 0.225], device=device) + torch.tensor(
            [0.485, 0.456, 0.406], device=device)
        pixel_vals_clamped = pixel_vals.clamp(0, 1) * 255
        r, g, b = pixel_vals_clamped.to(torch.int).tolist()
        abs_sum_gradients = abs_gradients_sum[y, x].item()
        rG, gG, bG = gradients[:, y, x].tolist()
        apple_prob = probabilities[0, 948].item()
        top_pixels_info.append((x.item(), y.item(), r, g, b, rG, gG, bG, r, g, b,  abs_sum_gradients, apple_prob))

    # Sort the top_pixels_info list by the gradient sum in descending order before returning
    top_pixels_info.sort(key=lambda x: x[8], reverse=True)
    # print(top_pixels_info)
    return top_pixels_info
# Load the image
img_path = '/content/timber_wolf.png'
img = Image.open(img_path)

# Function to calculate the range of influence for a single RGB value on the resized image

def can_rgb_fit(x, y, r, g, b, influence_range_matrix):
    # Check if the provided x and y are within the bounds of the matrix
    # if not (0 <= x < 256 and 0 <= y < 256):
    #     return False

    # Retrieve the list of influence ranges for the specified pixel
    influence_ranges = influence_range_matrix[x, y]
#     print((x,y,r,g,b))
#     print(influence_ranges)
#     quit()

    # Check if the provided RGB values fit within any of the influence ranges
    for lower_range, upper_range in influence_ranges:
        if (lower_range[0] <= r <= upper_range[0] and
            lower_range[1] <= g <= upper_range[1] and
            lower_range[2] <= b <= upper_range[2]):
            return True

    # If none of the ranges match, return False
    return False

def calculate_influence_range(original_rgb):
    # Calculate the influence range when changing the original value to 0 or 255
    # The change is divided by 9 because each pixel in the 3x3 block contributes 1/9th
    lower_change = -np.floor((original_rgb - np.array([0, 0, 0])) / 9).astype(int)
    upper_change = np.floor((np.array([255, 255, 255]) - original_rgb) / 9).astype(int)
    # Return the range as a tuple (lower_change, upper_change)
    return tuple(lower_change), tuple(upper_change)
# Let's take a look at the first few influence ranges for the top-left pixel
# influence_range_matrix[0, 0][:5]  # Showing only the first 5 for brevity

def optimize_pixel_modification(image_path, model, num_top_pixels=1000, rgb_range=28):


    # Initialize a 256x256 matrix, each cell contains a list of 9 tuples for the RGB influence range
    influence_range_matrix = np.empty((256, 256), dtype=object)

    # Iterate over the 256x256 resized image
    for i in range(256):
        for j in range(256):
            # Calculate the top-left corner of the corresponding 3x3 block in the original image
            block_top_left_i = i * 3
            block_top_left_j = j * 3

            # Initialize the list for this pixel in the resized image
            influence_range_matrix[i, j] = []

            # Iterate over the 3x3 block in the original image
            for di in range(3):
                for dj in range(3):
                    # Get the RGB value of the current pixel in the original image
                    original_pixel_rgb = img.getpixel((block_top_left_j + dj, block_top_left_i + di))
                    # Calculate the influence range for this pixel
                    influence_range = calculate_influence_range(np.array(original_pixel_rgb))
                    # Add the influence range to the list for the corresponding pixel in the resized image
                    influence_range_matrix[i, j].append(influence_range)

    now = time.time()
    image = Image.open(image_path).convert('RGB')
    resize = transforms.Resize((256, 256))
    image = resize(image)
    top_pixels = get_top_pixels_by_gradient_sum_sorted(image, model, top_x=num_top_pixels)

    best_probability = 0
    best_candidate = None
    itt = 0
    for pixel_info in top_pixels[214:300]:  # Iterate over the first 20 top pixels
        it = 0
        itt += 1
        x, y, r, g, b = map(int, pixel_info[:5])
        # Get the prediction probability for the current pixel
        current_pred = predict(pixel_info, image, model)
        current_probability = current_pred[-1]


        for dr in range(-rgb_range, rgb_range + 1):
            for dg in range(-rgb_range, rgb_range + 1):
                for db in range(-rgb_range, rgb_range + 1):
                    # Check if the new RGB values fit within the influence ranges
                    if can_rgb_fit(x, y, dr, dg, db, influence_range_matrix):
                        new_r, new_g, new_b = r + dr, g + dg, b + db
#                         print((r,g,b))
#                         print((dr,dg,db))
#                         print((new_r,new_g,new_b))
                        # Check if the modified RGB values are valid and within the range [0, 255]
                        if 0 <= new_r <= 255 and 0 <= new_g <= 255 and 0 <= new_b <= 255:
                            it += 1
#                             print(it)
                            new_pixel_pre = list(pixel_info[:2]) + [new_r, new_g, new_b] + list(pixel_info[5:])
                            new_pixel = predict(new_pixel_pre, image, model)
                            new_probability = new_pixel[-1]
                            if new_probability > best_probability and new_probability != 1337:
                                best_probability = new_probability
                                best_candidate = new_pixel
                                print(f"New best probability: {best_probability}, Best candidate: {best_candidate}")

        elapsed_time = time.time() - now
        print(f"{itt} Pixel done, optimization time: {elapsed_time}s Time per image {it / elapsed_time}  for total of {it} images at pixel {(x,y)}")
    return best_candidate


image_path = "/content/timber_wolf.png"
transform = transforms.ToTensor()
optimize_pixel_modification(image_path, model)

#[68, 108, 233, 254, 238, 0, 0, 0, 249, 249, 251, 0, 0.0006791171617805958]


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.16.0


New best probability: 0.0006531354156322777, Best candidate: [92, 133, 207, 207, 212, 0, 0, 0, 234, 234, 239, 0, 0.0006531354156322777]
New best probability: 0.0006531413528136909, Best candidate: [92, 133, 207, 207, 213, 0, 0, 0, 234, 234, 239, 0, 0.0006531413528136909]
New best probability: 0.000653146649710834, Best candidate: [92, 133, 207, 207, 214, 0, 0, 0, 234, 234, 239, 0, 0.000653146649710834]
New best probability: 0.0006531528779305518, Best candidate: [92, 133, 207, 207, 215, 0, 0, 0, 234, 234, 239, 0, 0.0006531528779305518]
New best probability: 0.0006531595136038959, Best candidate: [92, 133, 207, 207, 216, 0, 0, 0, 234, 234, 239, 0, 0.0006531595136038959]
New best probability: 0.0006531674298457801, Best candidate: [92, 133, 207, 207, 217, 0, 0, 0, 234, 234, 239, 0, 0.0006531674298457801]
New best probability: 0.0006531754625029862, Best candidate: [92, 133, 207, 207, 218, 0, 0, 0, 234, 234, 239, 0, 0.0006531754625029862]
New best probability: 0.0006531836115755141, Best 